<a href="https://colab.research.google.com/github/Bobbiejian/HelloWorld/blob/master/DisChoiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import numpy as np
import heapq
import os 
from scipy.special import comb, perm
from scipy.optimize import fsolve

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class RawDataset(object):
    def __init__(self,sizes,choices, size_list, count_map={}, subset_map={}):
        self.max_size = 5
        self.sizes=sizes # a np array
        self.choices=choices # a np array
        self.size_list = size_list
        self.item_count_map = count_map
        self.item_subset_map = subset_map        
        
        ut=utility()
        self.counts_map=ut.get_subset_counts(self)
        self.subsets, self.counts = ut.subsets_and_counts(self)
        
        # the number of unique item
        self.item_sum = self.getItemCount()
        
        self.itemSet, self.transList = self.getItemSetTransList()
    
    def getItemSetTransList(self):
        transList = list()
        itemSet = set()
        itor = self.iter_choices()
        for subset in itor:
            transaction = frozenset(subset[1])
            transList.append(transaction)
            for item in transaction:
                itemSet.add(frozenset([item]))  # Generate 1-itemSets
        return itemSet, transList
        
    def getItemCount(self):
        count = 0 
        for item in self.item_count_map:
            count += 1
        return count
    
        
        
    #1
    def iter_choices(self):
        curr_ind = 0
        choice_list = []
        
        for size in self.sizes:
            choice = self.choices[curr_ind:(curr_ind + size)]
            choice_list.append(choice)
            curr_ind += size
            
        if curr_ind != len(self.choices):
            print("it doesn't match up")
            
        return zip(self.sizes, choice_list)

In [6]:
class utility(object):
    def __init__(self):
        pass
    #1
    def vec2tuple(self,arr):
        return tuple(arr)
    #2
    # return the starting point of each subset in the dataset plus the last dimension is the whole size 
    def index_points(self,sizes): #sizes is an array of size of each subset in the dataset
        length = sizes.size
        inds = np.zeros(length+1, dtype=int)
        inds[1:length+1] = np.cumsum(sizes)
        return inds
    #3
    # get the count of each subset in the dataset,  data is a RawDataset
    def get_subset_counts(self,data):
        counts = {}
        inds = self.index_points(data.sizes)
        n=data.sizes.size
        for i in range(n):
            size = data.sizes[i] # number of the i-th subset
            choice = data.choices[inds[i]:inds[i+1]]
            choice_tup = self.vec2tuple(choice)
            
            # sort the tuple
            choice_tup = tuple(sorted(choice_tup))
            
            if counts.get(choice_tup,-1)<0:
                counts[choice_tup] = 1
            else:
                counts[choice_tup] += 1
        
        return counts
    
    #4
    # get the subsets as list of tuples and its corresponding size
    def subsets_and_counts(self,data):
        subsets = []
        counts = []
        dict_data = self.get_subset_counts(data)
        for subset, count in dict_data.items():
            subsets.append(subset)
            counts.append(count)
            
        return subsets, counts
            
    #5
    #read the data
    def read_data(self,fname):
        f = open(fname)
        item_count_map={}
        item_subset_map={}
        size_list={}
        
        # initialization of choices and sizes
        line = f.readline()
        wordlist = line.split()
        choices = np.array([int(wordlist[0])])
        wordlist.pop(0)
        
        for a in wordlist:
            choices = np.append(choices, int(a))
        sizes = np.array(choices.size)
        
        size_list[choices.size] = size_list.get(choices.size,0) + 1

        
        if choices.size > 1:
            choices = np.sort(choices)      
            for item in choices:
                item_count_map[item] = item_count_map.get(item,0)+1
                new_val = item_subset_map.get(item,[])
                #print(new_val)
                new_val.append(tuple(choices))
                item_subset_map[item] = new_val
                #print(item_subset_map[item])
        else:
            item = int(choices)
            item_count_map[item] = item_count_map.get(item,0)+1
            new_val = item_subset_map.get(item,[])
            #print(new_val)
            new_val.append(tuple(choices))
            item_subset_map[item] = new_val
                
        line = f.readline()
        while line:
            wordlist = line.split()
            tset = np.array([int(wordlist[0])])
            wordlist.pop(0)
            for a in wordlist:
                tset = np.append(tset, int(a))
                
            size_list[tset.size] = size_list.get(tset.size,0) + 1
            
            if tset.size > 1:
                tset = np.sort(tset)
            if tset.size < 6:
                choices = np.append(choices, tset)
                #print(tset)
                sizes = np.append(sizes, tset.size)
                
                if tset.size == 1:
                    item = int(tset)
                    item_count_map[item] = item_count_map.get(item,0)+1
                    new_val = item_subset_map.get(item,[])
                    #print(new_val)
                    new_val.append(tuple(tset))
                    item_subset_map[item] = new_val
                    
                else:
                    for item in tset:
                        item_count_map[item] = item_count_map.get(item,0)+1
                        new_val = item_subset_map.get(item,[])
                        #print(new_val)
                        new_val.append(tuple(tset))
                        item_subset_map[item] = new_val
                    
            line = f.readline()
        
        f.close()
        
        return RawDataset(sizes, choices, size_list, item_count_map, item_subset_map)
    
    #6 iterator over choices
    def iter_choices(self, data):
        curr_ind = 0
        choice_list = []
        
        for size in data.sizes:
            choice = data.choices[curr_ind:(curr_ind + size)]
            choice_list.append(choice)
            curr_ind += size
            
        if curr_ind != len(data.choices):
            print("it doesn't match up")
            
        return zip(data.sizes, choice_list)

In [7]:
class AssoRuleMining(object):
    def __init__(self,data):
        self.data = data
        self.itemSet, self.transList = self.getItemSetTransList()
        self.uni_count_map = self.getUniqueItemMap()
        
        
        
    def getUniqueItemMap(self):
        uni_item_count_map = {}

        
        for item in self.itemSet:
            for subset in self.transList:
                if item.issubset(subset):
                    uni_item_count_map[item] = uni_item_count_map.get(item,0) + 1
                    
        return uni_item_count_map
        
    
    def getItemSetTransList(self):
        transList = list()
        itemSet = set()
        itor = self.data.iter_choices()
        for subset in itor:
            transaction = frozenset(subset[1])
            transList.append(transaction)
            for item in transaction:
                itemSet.add(frozenset([item]))  # Generate 1-itemSets
        return itemSet, transList
    
    def returnItemsWithMinSupport(self, itemSet, transList, minSup, freqSet):
        """calculates the support for items in the itemSet and returns a subset
        of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = dict()

        for item in itemSet:
            for transaction in transList:
                if item.issubset(transaction):
                    freqSet[item] = freqSet.get(item,0) + 1
                    localSet[item] = localSet.get(item,0) + 1

        for item,value in localSet.items():
            supp = float(value) / len(transList)

            if supp >= minSup:
                _itemSet.add(item)

        return _itemSet
    
    def joinSet(self, itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

        
    def runMining(self, minSup, minPosConf, minNegaConf, rho):
        itemSet = self.itemSet
        transList = self.transList
        
        # Global dictionary which stores (key=n-itemSets,value=support)
        freqMap = dict()
        
        # Global dictionary which stores 
        # (key=size of itemSet,value=set of all itemSets which satisfy minSupport)
        largeMap = dict()

        assocRules = dict()
        # Dictionary which stores Association Rules
        
        #select out all the frequent single item and their support
        oneCSet = set()
#         for item_int in self.data.item_count_map:
#             item = frozenset([item_int])
#             freqMap[item] = self.data.item_count_map[item_int]
#             supp = self.data.item_count_map[item_int] / len(transList)
#             if supp >= minSup:
#                 oneCSet.add(item)

        for item in self.uni_count_map:
            freqMap[item] = self.uni_count_map[item]
            supp = freqMap[item] / len(transList)
            if supp >= minSup:
                oneCSet.add(item)
            

        curLSet = oneCSet
        k = 2
        
        while curLSet != set([]) and k < 4:
            largeMap[k - 1] = curLSet
            curLSet = self.joinSet(curLSet, k)
            curCSet = self.returnItemsWithMinSupport(curLSet, transList, minSup, freqMap)
            curLSet = curCSet
            k = k + 1

        def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqMap[item]) / len(transList)

        toRetItems = []
        candidSubset = []
        for key, value in largeMap.items():
            toRetItems.extend([(tuple(item), getSupport(item)) for item in value])
            candidSubset.extend([item for item in value])
        
        # print(candidSubset)


                
        # Dictionary to detect the duplicates in the mining
        dupSubset = {} 
        dupResult = {}
        posSet = []
        negaSet = []
        for aSet in candidSubset:
            for bSet in candidSubset:
                if aSet.isdisjoint(bSet) == False:
                    continue
                abSet = aSet.union(bSet)
                
                # dulication detection
                
                if dupResult.get(abSet,False) == True:
                    continue
                
                baTup = (bSet,aSet)
                abTup = (aSet,bSet)
                if dupSubset.get(baTup,False) == True or dupSubset.get(abTup,False) == True:
                    continue
                dupSubset[abTup] = True
                dupSubset[baTup] = True
                
                
                
                Sa = float(freqMap[aSet]) / len(transList)
                Sb = float(freqMap[bSet]) / len(transList)
                
                if freqMap.get(abSet,-1) > -1:
                    Sab = float(freqMap[abSet]) / len(transList)
                else:
                    Sab = self.calSupport(abSet, transList)
                
                corCoef = self.corCoef(Sa, Sb, Sab)
                
                # deal with postive correlation
                if corCoef > rho:
                    if Sab > minSup:
                        confCombined = np.sqrt(Sab**2 / (Sa*Sb))
                        if confCombined > minPosConf:
                            #posSet.append((abSet,corCoef,Sab,Sa,Sb,confCombined))
                            posSet.append(abSet)
                            dupResult[abSet] = True
                        
                        
                elif corCoef < -rho:
                    confCombined = np.sqrt((Sa-Sab)*(Sb-Sab)/(Sa*Sb))
                    if confCombined > minNegaConf:
                        #negaSet.append((abSet,corCoef,Sab,Sa,Sb,confCombined))
                        negaSet.append(abSet)                          
                        dupResult[abSet] = True
                        
                        
        return toRetItems, posSet, negaSet


    # Calculate the support of given subset(should be a frozenset)
    def calSupport(self, subset, transList):
        count = 0
        for transaction in transList:
            if subset.issubset(transaction):
                count += 1
        return count/len(transList)

    def corCoef(self,Sa,Sb,Sab):
        rho = (Sab - Sa*Sb)/np.sqrt(Sa*(1-Sa)*Sb*(1-Sb))
        #rho = (np.sqrt(abs(Sab - Sa*Sb)) * np.sign(Sab - Sa*Sb))/np.sqrt(Sa*(1-Sa)*Sb*(1-Sb))
        #print("The count %f %f %f " % (Sa,Sb,Sab))
        return rho


In [49]:
class ChoiceModel(object):
    def __init__(self, data, posSet, negaSet):
        self.data = data
        self.posSet = posSet
        self.negaSet = negaSet
        self.transList = self.data.transList
        self.card = len(self.transList)
        self.istrained = False
        
        # the key is int
        self.pItem = {}
        
        # the key is frozenset
        self.pPos = {}
        self.pNega = {}
        
        
        self.gamma = {}
        self.pos_count_map, self.pos_subset_map = self.assocSubsetMining(posSet)
        self.nega_count_map, self.nega_subset_map = self.assocSubsetMining(negaSet)
        self.z = self.findSizeRatio()
        
    def findSizeRatio(self):
        z = {}
        for key in self.data.size_list:
            z[key] = self.data.size_list[key] / len(self.data.transList)
        return z   
        
    def assocSubsetMining(self, assocList):
        # not using expanded data set, use frozenset instead
        rule_count_map = {}
        rule_subset_map = {}
        
        for ruleSet in assocList:
            #print("rule here")
            #print(ruleSet)
            for subset in self.data.iter_choices():
                #print(subset)
                #print("frozensubset!!!!!!1")
                #print(frozenset(subset[1]))
                #print(ruleSet)
                if ruleSet.issubset(frozenset(subset[1])):
                    rule_count_map[ruleSet] = rule_count_map.get(ruleSet,0) + 1
                    new_val = rule_subset_map.get(ruleSet,[]) 
                    new_val.append(subset[1])
                    rule_subset_map[ruleSet] = new_val 
                    
        return rule_count_map, rule_subset_map
    
    
    
    def iniAssocProb(self):
        
        for item, value in self.data.item_count_map.items():
            self.pItem[item] = value / sum(self.data.sizes)
        
        #for rule, value in self.pos_count_map.items():
        #    self.pPos[rule] = 0
        
        #for rule, value in self.nega_count_map.items():
        #    self.pNega[rule] = 0
            
        #for k in range(2, self.data.max_size+1):
        #    self.gamma[k] = 1

        aver_sup = len(self.transList) / (len(self.posSet) + len(self.negaSet))
        
        for rule, value in self.pos_count_map.items():
            self.pPos[rule] = (value - aver_sup) / aver_sup / self.data.item_sum ** len(rule)
        
        for rule, value in self.nega_count_map.items():
            self.pNega[rule] = (value - aver_sup) / aver_sup / self.data.item_sum ** len(rule)
    
            
        for size in range(2, self.data.max_size+1):
            self.gamma[size] = self.getGamma(size)
  
        print(self.pPos)
        print(self.pNega) 
            
        
    def getCorTerm(self,subset):
        trans = frozenset(subset)
        corItem = 0
        for pos in self.posSet:
            if pos.issubset(trans):
                corItem += self.pPos[pos]
                
        for nega in self.negaSet:
            if nega.issubset(trans):
                corItem += self.pNega[nega]
                
        return corItem
                
            
            
        
    def updateItemProb(self):
        # expanded data set is used here
        itemMap = self.data.item_subset_map
        sum_item = 0
        temp_map = {}
        
        # deal certain item
        for item, subList in itemMap.items():
            p_raw = 0
            
            # all the read data in expanded set
            for subset in subList:
                
                # single set
                if len(subset) == 1:
                    p_raw += 1
                
                # size > 2
                else:   
                    ps = 1
                    for item_temp in subset:
                        ps = ps * self.pItem[item_temp]
                        
                    p_raw += ps / (ps + self.getCorTerm(subset))
                    
            p_raw = p_raw / self.card
            
            #print(p_raw)
            
            # the whole possible set term
            whole_term = 0
            for k in range(2, self.data.max_size+1):
                temp_sum = 0
                for i in range(1,k+1):
                    temp_sum += i * comb(k,i) * self.pItem[item]**(i) * (1 - self.pItem[item])**(k-i)
                    #print(i * comb(k,i) * self.pItem[item]**(i) * (1 - self.pItem[item])**(k-i))
                    
                temp_sum = temp_sum * self.z.get(k,0) * self.gamma[k]
                #print(temp_sum)
                
                whole_term += temp_sum
            #print(item)
            #print(-whole_term)
            
            p_raw = p_raw - whole_term
            
            temp_map[item] = p_raw
            sum_item += p_raw
            
            
        for item, value in temp_map.items():
            self.pItem[item] = value / sum_item           
            
    def RHSsum(self,x,rule,ispos):
        if ispos == True:
            
            real_sum = 0
            for subset in self.pos_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset) - self.pPos[rule] + x)

            real_sum = real_sum / self.card

            whole_sum = 0
            
            for size in range(2, self.data.max_size+1):
                whole_sum += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)
        
        else:
            
            real_sum = 0
            for subset in self.nega_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset) - self.pNega[rule] + x)

            real_sum = real_sum / self.card

            whole_sum = 0
            
            for size in range(2, self.data.max_size+1):
                whole_sum += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)

        return real_sum - whole_sum
    
    def RHSsum_test(self,x,rule,ispos,istest):
        if ispos == True:
            
            real_sum = 0
            for subset in self.pos_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset) - self.pPos[rule] + x)

            real_sum = real_sum / self.card

            whole_sum = 0
            
            for size in range(2, self.data.max_size+1):
                whole_sum += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)
        
        else:
            
            real_sum = 0
            for subset in self.nega_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset) - self.pNega[rule] + x)

            real_sum = real_sum / self.card

            whole_sum = 0
            
            for size in range(2, self.data.max_size+1):
                whole_sum += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)

        print(real_sum)
        print(whole_sum)
        return real_sum , whole_sum , real_sum - whole_sum, real_sum - whole_sum / real_sum
    
    
    def getRuleGrad(self,rule,ispos):
        if ispos == True:
            
            real_sum = 0
            for subset in self.pos_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset))

            real_sum = real_sum / self.card

        
        else:
            
            real_sum = 0
            for subset in self.nega_subset_map[rule]:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]

                real_sum += 1 / (ps + self.getCorTerm(subset))

            real_sum = real_sum / self.card

        return real_sum
                
           
    def updateAssocProbAndGamma(self,l_rate):
        temp_pos = {}
        temp_nega = {}
        temp_gamma = {}
        
        # deal with positive rule
        #for rule in self.pos_subset_map:
            #temp_pos[rule] = fsolve(self.RHSsum, self.pPos[rule], args=(rule,True))[0]
   
        
        #deal with negative rule  
        #for rule in self.nega_subset_map:
            #temp_nega[rule] = fsolve(self.RHSsum, self.pNega[rule], args=(rule,False))[0]
            
        # deal with positive rule
        for rule in self.pos_subset_map:
            temp_pos[rule] = self.pPos[rule] + l_rate * self.getRuleGrad(rule, True)
   
        
        #deal with negative rule  
        for rule in self.nega_subset_map:
            temp_nega[rule] = self.pNega[rule] + l_rate * self.getRuleGrad(rule, False)

            
        self.pPos = temp_pos
        self.pNega = temp_nega
        
        # deal with gamma, using the new pPos and pNega
        for size in range(2, self.data.max_size+1):
            self.gamma[size] = self.getGamma(size)
            
            
    def getGamma(self,k):
        sum_pos = 0
        for rule in self.posSet:
            sum_pos += self.getAssocComb(k,rule) * self.pPos[rule]
        
        sum_nega = 0
        for rule in self.negaSet:
            sum_nega += self.getAssocComb(k,rule) * self.pNega[rule]
            
        denom = sum_pos + sum_nega + 1
        
        return 1/denom
        
            
    def getAssocComb(self,k,rule):
        lr = len(rule)
        card = self.data.item_sum
        
        start = perm(k, lr) * (card-lr)**(k-lr)
        
        if k < lr:
            return 0
            
        if k == lr:
            return start
        
        if (k-lr)==1:
            start = start + comb(lr,1) * perm(k, k) / perm(2,2) 
            
        if (k-lr)==2:
            start = start + comb(lr,1) * perm(k, k) / perm(2,2) * (card-lr) \
                    + comb(lr,2) * perm(k, k) / (perm(2,2)*perm(2,2)) \
                    + comb(lr,1) * perm(k, k) / perm(3,3)
            
        if (k-lr)==3:
            start = start + comb(lr,1) * perm(k, k) / perm(2,2) * (card-lr)**2 \
                    + comb(lr,2) * perm(k, k) * (card-lr) / (perm(2,2)*perm(2,2)) \
                    + comb(lr,1) * perm(k, k) * (card-lr) / perm(3,3) \
                    + comb(lr,3) * perm(k, k) / perm(2,2)**3 \
                    + comb(lr,2) * 2 * perm(k, k) / perm(2,2) / perm(3,3) \
                    + comb(lr,1) * perm(k, k) / perm(4,4)
        return start
    
    
    def train(self, tol=0.001):
        # Initialize all the p of rules to be 1, and gamma is also 1
        self.iniAssocProb()
        
        #print(self.pNega)
        #print(self.nega_count_map)
        
        # Initialize all the p of items to be proportional to support
        self.updateItemProb()
#         sum_item = 0
#         for item, value in self.data.item_count_map.items():
#             self.pItem[item] = value / len(self.transList)
        
        print(self.pItem)
        self.updateAssocProbAndGamma(0.1)

        #curpath = os.getcwd()
        #file = curpath + "/read_test_uni.txt"
        # 判断文件是否存在，不存在则创建

        #f = open(file)
        for rule in self.posSet:
            test = self.RHSsum_test(self.pPos[rule],rule,True,True)
            print(str([i for i in rule]))
            print(str(test))
            
        for rule in self.negaSet:
            test = self.RHSsum_test(self.pNega[rule],rule,False,True)
            print(str([i for i in rule]))
            print(str(test))


        iter = 100
        for i in range(iter):
            self.updateItemProb()
            self.updateAssocProbAndGamma(0.1)

        print("Iteration ::: :::::::::::" )
        print(str(iter))
        #f.writelines(self.pNega)
        for rule in self.posSet:
            test = self.RHSsum_test(self.pPos[rule],rule,True,True)
            print(str([i for i in rule]))
            print(str(test))
            
        for rule in self.negaSet:
            test = self.RHSsum_test(self.pNega[rule],rule,False,True)
            print(str([i for i in rule]))            
            print(str(test))

        
            
        # Update all the p of rules and gamma according to KKT stationary condition and pItem above
        
        

In [55]:

nn

{frozenset({5, 40}): -0.14398897543723665,
 frozenset({3, 40}): 0.06868633999959928,
 frozenset({14, 24, 40}): 0.03236072158917575,
 frozenset({5, 32}): 0.053092986740367026,
 frozenset({3, 32}): 0.45390682657997294,
 frozenset({5, 14}): 0.059436979140478505,
 frozenset({3, 14}): 0.04078550644854294,
 frozenset({5, 24}): -0.04564522455039294,
 frozenset({3, 24}): 0.06572433953148733,
 frozenset({3, 8}): 0.06625505627354689,
 frozenset({8, 14, 24}): 0.03971176276824466,
 frozenset({5, 45}): -0.04486008586809379,
 frozenset({3, 45}): 0.3220600776919364,
 frozenset({14, 24, 45}): 0.024102425542268876,
 frozenset({5, 16}): 0.03439156633147719,
 frozenset({3, 16}): 0.9321726985928375,
 frozenset({3, 41}): 0.14138037891439548,
 frozenset({3, 48}): -0.3646754331466669,
 frozenset({3, 11}): -0.1255133664448298,
 frozenset({5, 9}): 0.12457614764662527,
 frozenset({5, 46}): -0.06119832741082827,
 frozenset({3, 46}): 1.3115620265627714,
 frozenset({5, 12}): 0.13424154317419545,
 frozenset({5, 34}

In [54]:
modelk.pNega

{frozenset({5, 40}): -0.2269759402568966,
 frozenset({3, 40}): 0.10087938929893477,
 frozenset({14, 24, 40}): 0.03430442691114385,
 frozenset({5, 32}): 0.13228490577034796,
 frozenset({3, 32}): 0.47971825040218297,
 frozenset({5, 14}): 0.14207006526749721,
 frozenset({3, 14}): 0.06658090540411622,
 frozenset({5, 24}): -0.11755276352218562,
 frozenset({3, 24}): 0.1313745335119438,
 frozenset({3, 8}): 0.0955679287277366,
 frozenset({8, 14, 24}): 0.04176406690276,
 frozenset({5, 45}): -0.11396099275550965,
 frozenset({3, 45}): 0.3453507790193765,
 frozenset({14, 24, 45}): 0.025590160496249063,
 frozenset({5, 16}): 0.09033275659633651,
 frozenset({3, 16}): 0.9479184857544851,
 frozenset({3, 41}): 0.1354821290275754,
 frozenset({3, 48}): -0.41061574988363336,
 frozenset({3, 11}): -0.16549767191232936,
 frozenset({5, 9}): 0.1845516984128605,
 frozenset({5, 46}): -0.1463997439065301,
 frozenset({3, 46}): 1.3214605864926403,
 frozenset({5, 12}): 0.19996933388782914,
 frozenset({5, 34}): -0.149

In [50]:
ut = utility()
os.chdir('/content/drive/MyDrive/DSC_exp/')
fname= os.getcwd() + '/bakery-5-25.txt'
datar = ut.read_data(fname)
p=AssoRuleMining(datar)
i,pos,nega = p.runMining(0.025,0.1,0.95,0.06)
modelk = ChoiceModel(datar,pos,nega)
modelk.train()

{frozenset({40, 46}): 0.0013933854907539117, frozenset({32, 15}): 0.0008599573257467993, frozenset({32, 34}): 0.0011592105263157893, frozenset({32, 34, 15}): 1.6089615931721196e-05, frozenset({43, 47}): 0.0008930298719772403, frozenset({25, 10}): 0.0005772403982930298, frozenset({48, 10}): 0.0005761735419630155, frozenset({48, 25, 10}): 1.0381934566145093e-05, frozenset({48, 25}): 0.0005751066856330014, frozenset({1, 5}): 0.0006615220483641536, frozenset({1, 4}): 0.0008791607396870553, frozenset({1, 3}): 0.0008903627311522048, frozenset({1, 4, 5}): 1.2014224751066855e-05, frozenset({1, 3, 4}): 1.650568990042674e-05, frozenset({1, 3, 5}): 1.2035561877667141e-05, frozenset({18, 3}): 0.0008711593172119487, frozenset({18, 29}): 0.0006289829302987197, frozenset({18, 3, 29}): 1.1534139402560454e-05, frozenset({38, 30}): 0.0005735064011379799, frozenset({20, 30}): 0.0005703058321479374, frozenset({21, 30}): 0.0005761735419630155, frozenset({38, 21, 30}): 1.0488620199146515e-05, frozenset({20,

In [ ]:
modelk.pNega

In [ ]:
ut = utility()
os.chdir('/content/drive/MyDrive/DSC_exp/')
fname= os.getcwd() + '/bakery-5-25.txt'
datar = ut.read_data(fname)
p=AssoRuleMining(datar)
i,pos,nega = p.runMining(0.025,0.1,0.95,0.06)
modelr = ChoiceModel(datar,pos,nega)
modelr.train()

In [51]:
modelr.pPos


{frozenset({40, 46}): 3.4984785450842835,
 frozenset({15, 32}): 2.154289101605498,
 frozenset({32, 34}): 2.9055019832359585,
 frozenset({15, 32, 34}): 2.014901468657894,
 frozenset({43, 47}): 2.24869988433563,
 frozenset({10, 25}): 1.4487256684004535,
 frozenset({10, 48}): 1.445967437076311,
 frozenset({25, 48}): 1.4433730933553681,
 frozenset({10, 25, 48}): 1.3023241671336547,
 frozenset({1, 5}): 1.6564414153557254,
 frozenset({1, 4}): 2.200695530907977,
 frozenset({1, 3}): 2.229926225353986,
 frozenset({1, 4, 5}): 1.5032062840294018,
 frozenset({1, 3, 4}): 2.0653710595463646,
 frozenset({1, 3, 5}): 1.5058771124018366,
 frozenset({3, 18}): 2.186527884861143,
 frozenset({18, 29}): 1.577427524737487,
 frozenset({3, 18, 29}): 1.445862182688041,
 frozenset({30, 38}): 1.4363237970613787,
 frozenset({20, 30}): 1.42833041425759,
 frozenset({21, 30}): 1.4430494541373042,
 frozenset({21, 30, 38}): 1.312792459711297,
 frozenset({20, 21, 30}): 1.3154640871196475,
 frozenset({20, 30, 38}): 1.3127

In [49]:

g

{frozenset({5, 40}): -0.6936440680151337,
 frozenset({3, 40}): -0.6812276445983875,
 frozenset({14, 24, 40}): -0.9372685886732282,
 frozenset({5, 32}): -0.786573932749577,
 frozenset({3, 32}): -0.7764968209465503,
 frozenset({5, 14}): -0.7824841787514258,
 frozenset({3, 14}): -0.785200679722075,
 frozenset({5, 24}): -0.7826161063483571,
 frozenset({3, 24}): -0.7665556632701916,
 frozenset({3, 8}): -0.702626377586541,
 frozenset({8, 14, 24}): -0.9358902966591623,
 frozenset({5, 45}): -0.7347080926441054,
 frozenset({3, 45}): -0.7154547621849893,
 frozenset({14, 24, 45}): -0.9452679292319368,
 frozenset({5, 16}): -0.7132820125800352,
 frozenset({3, 16}): -0.7038301874266711,
 frozenset({3, 41}): -0.8050627317534431,
 frozenset({3, 48}): -0.8529167877824353,
 frozenset({3, 11}): -0.8189656573458292,
 frozenset({5, 9}): -0.7784137090157909,
 frozenset({5, 46}): -0.7333794649451821,
 frozenset({3, 46}): -0.7214243508045111,
 frozenset({5, 12}): -0.8223987119574531,
 frozenset({5, 34}): -0.7

In [12]:
fname= os.getcwd() + '/read_test.txt'
data2 = ut.read_data(fname)
print(data2.item_count_map)
p=AssoRuleMining(data2)
print(p.uni_count_map)
i,pos,nega=p.runMining(0.2,0.05,0.05,0.3)
print(data2.item_subset_map)
print(p.transList)

model2 = ChoiceModel(data2,pos,nega)
print(model2.pos_count_map)
print(model2.pos_subset_map)
len(data2.item_subset_map[4][2])
data2.item_subset_map[4][3][1]
print(data2.sizes)
data2.item_count_map
print(model2.z)

NameError: ignored

In [ ]:
fname= os.getcwd() + '/read_test_uni.txt'
data2 = ut.read_data(fname)
p=AssoRuleMining(data2)
p.runMining(0.2,0.05,0.05,0.3)

([((2,), 0.8571428571428571),
  ((1,), 0.42857142857142855),
  ((4,), 0.7142857142857143),
  ((2, 4), 0.5714285714285714),
  ((1, 2), 0.42857142857142855)],
 [(frozenset({1, 2}),
   0.3535533905932738,
   0.42857142857142855,
   0.8571428571428571,
   0.42857142857142855,
   0.7071067811865476)],
 [(frozenset({1, 4}),
   -0.7302967433402215,
   0.14285714285714285,
   0.42857142857142855,
   0.7142857142857143,
   0.7302967433402214),
  (frozenset({1, 2, 4}),
   -0.4166666666666666,
   0.14285714285714285,
   0.42857142857142855,
   0.5714285714285714,
   0.7071067811865476)])

In [ ]:
l=ChoiceModel(data2,[frozenset([1,2])],[])
print(l.pItem)
l.train()
print(l.pItem)
print(l.transList)
print(l.pos_count_map)
l.z

{}
{1: 0.42857142857142855, 2: 0.8571428571428571, 3: 0.14285714285714285, 4: 0.7142857142857143}
[frozenset({1, 2, 3, 4}), frozenset({2, 4}), frozenset({1, 2}), frozenset({4}), frozenset({2, 4}), frozenset({1, 2}), frozenset({2, 4})]
{frozenset({1, 2}): 3}


{4: 0.14285714285714285, 2: 0.7142857142857143, 1: 0.14285714285714285}

In [ ]:
l={}
l[2] = 2
l[3] = 3
class test(object):
    def func(self,x,b,c):
        return (x - b - l[c])
    
    def te(self):
        return fsolve(func,5,args=(5,3))

t = test()
t.te()

array([8.])

In [ ]:
def updateAssocProbAndGamma(self):
        temp_pos = {}
        temp_nega = {}
        temp_gamma = {}
        
        # deal with positive rule
        for rule, value in self.pos_subset_map.items():
            nominator = 0
            for subset in value:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]
                    
                nominator += 1 / (ps + self.getCorTerm(subset))
                
            nominator = nominator / self.card
            
            denominator = 0
            for size in range(2, self.data.max_size+1):
                denominator += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)
                
            temp_pos[rule] = nominator / denominator
        
        #deal with negative rule    
        for rule, value in self.nega_subset_map.items():
            nominator = 0
            for subset in value:
                ps = 1
                for item_temp in subset:
                    ps = ps * self.pItem[item_temp]
                    
                nominator += 1 / (ps + self.getCorTerm(subset))
                
            nominator = nominator / self.card
            
            denominator = 0
            for size in range(2, self.data.max_size+1):
                denominator += self.z.get(size,0) * self.gamma[size] * self.getAssocComb(size,rule)
                
            temp_nega[rule] = nominator / denominator
            